In [47]:
from typing import Dict, Optional, Sequence, Tuple

from google.cloud import aiplatform
from google.cloud.aiplatform import explain

import tensorflow as tf

from keras import layers
from keras.layers import TextVectorization
from keras.models import load_model 

In [134]:
project_id='ihr-vertex-pipelines'
my_region='europe-west4' # :flag-nl:
model_name='model_text_jan27'
endpoint_name='raw_model_endpoint'
model_location='gs://ihr-vertex-pipelines/0.13+14.gb9a60f7/batch=8192/epochs=15/model'

In [10]:
def upload_model(
    project: str,
    location: str,
    display_name: str,
    serving_container_image_uri: str,
    artifact_uri: Optional[str] = None,
    serving_container_predict_route: Optional[str] = None,
    serving_container_health_route: Optional[str] = None,
    description: Optional[str] = None,
    serving_container_command: Optional[Sequence[str]] = None,
    serving_container_args: Optional[Sequence[str]] = None,
    serving_container_environment_variables: Optional[Dict[str, str]] = None,
    serving_container_ports: Optional[Sequence[int]] = None,
    instance_schema_uri: Optional[str] = None,
    parameters_schema_uri: Optional[str] = None,
    prediction_schema_uri: Optional[str] = None,
    explanation_metadata: Optional[explain.ExplanationMetadata] = None,
    explanation_parameters: Optional[explain.ExplanationParameters] = None,
    sync: bool = True,
):

    aiplatform.init(project=project, location=location)

    model = aiplatform.Model.upload(
        display_name=display_name,
        artifact_uri=artifact_uri,
        serving_container_image_uri=serving_container_image_uri,
        serving_container_predict_route=serving_container_predict_route,
        serving_container_health_route=serving_container_health_route,
        instance_schema_uri=instance_schema_uri,
        parameters_schema_uri=parameters_schema_uri,
        prediction_schema_uri=prediction_schema_uri,
        description=description,
        serving_container_command=serving_container_command,
        serving_container_args=serving_container_args,
        serving_container_environment_variables=serving_container_environment_variables,
        serving_container_ports=serving_container_ports,
        explanation_metadata=explanation_metadata,
        explanation_parameters=explanation_parameters,
        sync=sync,
    )

    model.wait()

    print(model.display_name)
    print(model.resource_name)
    return model


## Upload and deploy raw model

This is just a raw deploy of the `saved_model.pb`, so using this model requires using the transformed data in TF-IDF format (or whatever the model was trained with). What we want is a model that takes the text of a review. This requires:

* Including the `transform_fn` somehow
* Probably [specifying schemas](https://cloud.google.com/vertex-ai/docs/reference/rest/v1beta1/projects.locations.models?&_ga=2.51426537.-1793164395.1642757755#predictschemata), predict route? Deploying a custom container?

In [11]:
model = upload_model(
    project=project_id,
    location=my_region,
    display_name=model_name,
    serving_container_image_uri='europe-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-7:latest',
    artifact_uri=model_location + '/saved_model/'
)

INFO:google.cloud.aiplatform.models:Creating Model
INFO:google.cloud.aiplatform.models:Create Model backing LRO: projects/237148598933/locations/europe-west4/models/2258080224103104512/operations/3153930412577783808
INFO:google.cloud.aiplatform.models:Model created. Resource name: projects/237148598933/locations/europe-west4/models/2258080224103104512
INFO:google.cloud.aiplatform.models:To use this Model in another session:
INFO:google.cloud.aiplatform.models:model = aiplatform.Model('projects/237148598933/locations/europe-west4/models/2258080224103104512')
model_text_jan27
projects/237148598933/locations/europe-west4/models/2258080224103104512


Check out deployed models: [deployed models in console](https://pantheon2.corp.google.com/vertex-ai/models?project=ihr-vertex-pipelines)

In [ ]:
def create_endpoint(
    project: str, display_name: str, location: str,
):
    aiplatform.init(project=project, location=location)

    endpoint = aiplatform.Endpoint.create(
        display_name=display_name, project=project, location=my_region,
    )

    print(endpoint.display_name)
    print(endpoint.resource_name)
    return endpoint



In [ ]:
# Create it
endpoint = create_endpoint(project_id, endpoint_name, my_region)

In [17]:
# Or get an existing one
endpoint = aiplatform.Endpoint('5446488022793060352')

In [18]:
endpoint_id = endpoint.resource_name.split('/')[-1]
endpoint_id
#endpoint_name

'5446488022793060352'

Check out model endpoints: [model endpoints in console](https://pantheon2.corp.google.com/vertex-ai/endpoints?project=ihr-vertex-pipelines)

In [19]:
def deploy_model_with_dedicated_resources(
    project,
    location,
    model_name: str,
    machine_type: str,
    endpoint: Optional[aiplatform.Endpoint] = None,
    deployed_model_display_name: Optional[str] = None,
    traffic_percentage: Optional[int] = 0,
    traffic_split: Optional[Dict[str, int]] = None,
    min_replica_count: int = 1,
    max_replica_count: int = 1,
    accelerator_type: Optional[str] = None,
    accelerator_count: Optional[int] = None,
    explanation_metadata: Optional[explain.ExplanationMetadata] = None,
    explanation_parameters: Optional[explain.ExplanationParameters] = None,
    metadata: Optional[Sequence[Tuple[str, str]]] = (),
    sync: bool = True,
):
    """
        model_name: A fully-qualified model resource name or model ID.
              Example: "projects/123/locations/us-central1/models/456" or
              "456" when project and location are initialized or passed.
    """

    aiplatform.init(project=project, location=location)

    model = aiplatform.Model(model_name=model_name)

    # The explanation_metadata and explanation_parameters should only be
    # provided for a custom trained model and not an AutoML model.
    model.deploy(
        endpoint=endpoint,
        deployed_model_display_name=deployed_model_display_name,
        traffic_percentage=traffic_percentage,
        traffic_split=traffic_split,
        machine_type=machine_type,
        min_replica_count=min_replica_count,
        max_replica_count=max_replica_count,
        accelerator_type=accelerator_type,
        accelerator_count=accelerator_count,
        explanation_metadata=explanation_metadata,
        explanation_parameters=explanation_parameters,
        metadata=metadata,
        sync=sync, # Whether to execute this method synchronously
    )

    model.wait()

    print(model.display_name)
    print(model.resource_name)
    return model


In [21]:
model_name

'model_text_jan27'

In [22]:
model

resource name: projects/237148598933/locations/europe-west4/models/2258080224103104512

In [24]:
deployed_model = deploy_model_with_dedicated_resources(
    project_id, 
    my_region, 
    model.resource_name, 
    'n1-standard-4', 
    endpoint, 
    traffic_percentage=100)

INFO:google.cloud.aiplatform.models:Deploying model to Endpoint : projects/237148598933/locations/europe-west4/endpoints/5446488022793060352
INFO:google.cloud.aiplatform.models:Deploy Endpoint model backing LRO: projects/237148598933/locations/europe-west4/endpoints/5446488022793060352/operations/5081471053092356096
INFO:google.cloud.aiplatform.models:Endpoint model deployed. Resource name: projects/237148598933/locations/europe-west4/endpoints/5446488022793060352
model_text_jan27
projects/237148598933/locations/europe-west4/models/2258080224103104512


## Get some predictions from the deployed model using test data

In [25]:
def endpoint_predict(
    project: str, location: str, instances: list, endpoint: str
):
    aiplatform.init(project=project, location=location)

    endpoint = aiplatform.Endpoint(endpoint)

    prediction = endpoint.predict(instances=instances)
    print(prediction)
    return prediction

In [135]:
restored_model = load_model(model_location + '/vectorizer/')
restored_vectorizer = restored_model.layers[0]
restored_vectorizer.get_config()

{'name': 'text_vectorization',
 'trainable': True,
 'batch_input_shape': (None,),
 'dtype': 'string',
 'max_tokens': 20000,
 'standardize': 'lower_and_strip_punctuation',
 'split': 'whitespace',
 'ngrams': 2,
 'output_mode': 'multi_hot',
 'output_sequence_length': None,
 'pad_to_max_tokens': False,
 'sparse': False,
 'ragged': False,
 'vocabulary': None,
 'idf_weights': None}

In [138]:
testdata = 'gs://ihr-vertex-pipelines/data/prepared/test/test_data-00000-of-00001.tfrecord'
raw_dataset = tf.data.TFRecordDataset([testdata])
raw_dataset

<TFRecordDatasetV2 shapes: (), types: tf.string>

In [142]:
# not sure why this doesn't work
kerasmodel.predict(raw_dataset.map(restored_vectorizer))

ValueError: in user code:

    File "/opt/conda/lib/python3.7/site-packages/keras/engine/training.py", line 1621, in predict_function  *
        return step_function(self, iterator)
    File "/opt/conda/lib/python3.7/site-packages/keras/engine/training.py", line 1611, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/opt/conda/lib/python3.7/site-packages/keras/engine/training.py", line 1604, in run_step  **
        outputs = model.predict_step(data)
    File "/opt/conda/lib/python3.7/site-packages/keras/engine/training.py", line 1572, in predict_step
        return self(x, training=False)
    File "/opt/conda/lib/python3.7/site-packages/keras/utils/traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/opt/conda/lib/python3.7/site-packages/keras/engine/input_spec.py", line 227, in assert_input_compatibility
        raise ValueError(f'Input {input_index} of layer "{layer_name}"

In [147]:
# go to numpy and that works
instances = np.array([x.numpy() for x in raw_dataset.map(restored_vectorizer).take(5)])

In [146]:
kerasmodel.predict(instances)

array([[0.09841356],
       [0.37950382],
       [0.24578968],
       [0.16459799],
       [0.36051244]], dtype=float32)

In [148]:
# but then why doesn't this?
endpoint_id = '5446488022793060352'
pred = endpoint_predict(project_id, my_region, instances, endpoint_id)
pred

ValueError: Unable to coerce value: array([1., 1., 1., ..., 0., 0., 0.], dtype=float32)

In [29]:
raw_instances[0].features.feature['text'].bytes_list.value[0]

b'************* SPOILERS BELOW ************* "\'Night, Mother" is the story of Jesse (Sissy Spacek), a divorced epileptic woman who calmly announces to her brash mother (Anne Bancroft) that she\'s going to commit suicide. This is a fascinating premise that is drained of all vitality and excitement. The brilliant hook turns out to be a cheat- the story that follows is lacking in substance, gravity and revelatory value. Where are the shocks and surprises as mother and daughter have what may be the last conversation of their lives? Where are the secrets revealed, the confessions and fantasies and regrets? They\'re here, but they\'ve all been painted the same dull color that keeps emotion in the background and celebrates the \'genius\' of playwright Marsha Norman at the expense of everything else. The result is not a film but an exhausting endurance test.<br /><br />Let me preface my comments by saying I find Sissy Spacek to be one of the greatest actresses in the history of motion picture

In [30]:
def to_instance(raw_record, key):
    text_bytes = raw_record.features.feature['text'].bytes_list.value[0]
    return { 'values': text_bytes, 'key': key }

In [74]:
instances = {'instances': [to_instance(v, k) for k, v in enumerate(raw_instances)]}

instances = [ list(vectorizer(test_text).numpy()) ] # should be: list of examples, each example is a vector that is the output of the Vectorizer after processing a review

#instances

## Test saving/loading Vectorization

In [43]:
vectorizer: TextVectorization = layers.TextVectorization(ngrams=2, max_tokens=20000, output_mode="multi_hot")
vectorizer.adapt(raw_dataset)

In [46]:
model = tf.keras.models.Sequential()
model.add(tf.keras.Input(shape=(1,), dtype=tf.string))
model.add(vectorizer)

model.save('gs://ihr-vertex-pipelines/tmp/vectorizer/')

2022-01-28 16:02:41.313806: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: gs://ihr-vertex-pipelines/tmp/vectorizer/assets


In [48]:
restored_model = load_model('gs://ihr-vertex-pipelines/tmp/vectorizer/')
restored_vectorizer = restored_model.layers[0]

In [53]:
test_text = "The quick brown fox jumps over the lazy dog"
vectorizer(test_text).numpy()[:50]

array([1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
      dtype=float32)

In [61]:
restored_vectorizer(test_text)

<tf.Tensor: shape=(20000,), dtype=float32, numpy=array([1., 1., 0., ..., 0., 0., 0.], dtype=float32)>

In [63]:
restored_vectorizer(test_text).numpy()[:50]

array([1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
      dtype=float32)

In [65]:
vectorizer.get_config()

{'name': 'text_vectorization_2',
 'trainable': True,
 'batch_input_shape': (),
 'dtype': 'string',
 'max_tokens': 20000,
 'standardize': 'lower_and_strip_punctuation',
 'split': 'whitespace',
 'ngrams': 2,
 'output_mode': 'multi_hot',
 'output_sequence_length': None,
 'pad_to_max_tokens': False,
 'sparse': False,
 'ragged': False,
 'vocabulary': None,
 'idf_weights': None}

In [66]:
restored_vectorizer.get_config()

{'name': 'text_vectorization_2',
 'trainable': True,
 'batch_input_shape': (),
 'dtype': 'string',
 'max_tokens': 20000,
 'standardize': 'lower_and_strip_punctuation',
 'split': 'whitespace',
 'ngrams': 2,
 'output_mode': 'multi_hot',
 'output_sequence_length': None,
 'pad_to_max_tokens': False,
 'sparse': False,
 'ragged': False,
 'vocabulary': None,
 'idf_weights': None}

## Raw model load and test

In [ ]:
kerasmodel = load_model(model_location + 'saved_model/')

In [84]:
kerasmodel.get_config()

{'name': 'my-kschool-model',
 'layers': [{'class_name': 'InputLayer',
   'config': {'batch_input_shape': (None, 20000),
    'dtype': 'float32',
    'sparse': False,
    'ragged': False,
    'name': 'input_1'},
   'name': 'input_1',
   'inbound_nodes': []},
  {'class_name': 'Dense',
   'config': {'name': 'dense',
    'trainable': True,
    'dtype': 'float32',
    'units': 16,
    'activation': 'relu',
    'use_bias': True,
    'kernel_initializer': {'class_name': 'GlorotUniform',
     'config': {'seed': None}},
    'bias_initializer': {'class_name': 'Zeros', 'config': {}},
    'kernel_regularizer': None,
    'bias_regularizer': None,
    'activity_regularizer': None,
    'kernel_constraint': None,
    'bias_constraint': None},
   'name': 'dense',
   'inbound_nodes': [[['input_1', 0, 0, {}]]]},
  {'class_name': 'Dropout',
   'config': {'name': 'dropout',
    'trainable': True,
    'dtype': 'float32',
    'rate': 0.5,
    'noise_shape': None,
    'seed': None},
   'name': 'dropout',
   'i

In [139]:
input_tensor = vectorizer("bad")
reviews = ["great movie", "bad movie", "sucks"]
vects = [restored_vectorizer(x).numpy() for x in reviews]

# convert input to numpy array and put into list (2-dim array actually)
examples = np.array(vects)
kerasmodel.predict(examples)

array([[0.5589698 ],
       [0.43218794],
       [0.46867135]], dtype=float32)

In [120]:
vectorized_input = raw_dataset.map(lambda x: vectorizer(x))
vectorized_input

<MapDataset shapes: (20000,), types: tf.float32>

In [125]:
kerasmodel.predict(vectorized_input)

ValueError: in user code:

    File "/opt/conda/lib/python3.7/site-packages/keras/engine/training.py", line 1621, in predict_function  *
        return step_function(self, iterator)
    File "/opt/conda/lib/python3.7/site-packages/keras/engine/training.py", line 1611, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/opt/conda/lib/python3.7/site-packages/keras/engine/training.py", line 1604, in run_step  **
        outputs = model.predict_step(data)
    File "/opt/conda/lib/python3.7/site-packages/keras/engine/training.py", line 1572, in predict_step
        return self(x, training=False)
    File "/opt/conda/lib/python3.7/site-packages/keras/utils/traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/opt/conda/lib/python3.7/site-packages/keras/engine/input_spec.py", line 227, in assert_input_compatibility
        raise ValueError(f'Input {input_index} of layer "{layer_name}"

In [136]:
reviews = ["great movie", "bad movie", "sucks"]
vects = [vectorizer(x) for x in reviews]
tf.data.Dataset.from_tensor_slices((vects))
#kerasmodel.predict()

<TensorSliceDataset shapes: (20000,), types: tf.float32>